## 2019136011 권성민

### 요구사항 1. titanic_dataset.py 분석

In [78]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

File path 얻기 위해 os package import
파일을 읽기 위해 또 전처리를 위해 pandas package import
deep learning (tensor) 연산을 위해 torch import
torch에 맞는 데이터를 불러오기 위해서 torch.utils.data import

In [79]:
class TitanicDataset(Dataset):
  def __init__(self, X, y):
    self.X = torch.FloatTensor(X)
    self.y = torch.LongTensor(y)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    target = self.y[idx]
    return {'input': feature, 'target': target}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str

Titanic Dataset을 저장하기 위한 클래스로, torch.utils.data.Dataset을 상속한다.

생성자에서 feature 배열 X와 target 배열 y를 받아, 멤버변수 X,Y에 저장한다.

데이터 길이를 저장하기 위해서 __len__을 override한다.

torch에서 순서대로 데이터와 결과를 갖고 오기 위해 __getitem__을 override한다. 이때 데이터는 입력된 index에 맞게 반환한다.

__str__을 override하여, 디버깅을 편리하게 한다.

In [80]:
class TitanicTestDataset(Dataset):
  def __init__(self, X):
    self.X = torch.FloatTensor(X)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    return {'input': feature}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str

Test를 위한 Dataset 클래스로, feature만 입력 받는다. 각 override는 위 클래스와 동일하고, target만 받지 않는다는 점만 다르다

In [81]:
def get_preprocessed_dataset():
    CURRENT_FILE_PATH = os.path.dirname(os.path.abspath("__file__"))

    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    all_df = pd.concat([train_df, test_df], sort=False)

    all_df = get_preprocessed_dataset_1(all_df)

    all_df = get_preprocessed_dataset_2(all_df)

    all_df = get_preprocessed_dataset_3(all_df)

    all_df = get_preprocessed_dataset_4(all_df)

    all_df = get_preprocessed_dataset_5(all_df)

    all_df = get_preprocessed_dataset_6(all_df)

    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset)
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

현재 파일이 있는 디렉터리의 경로를 CURRENT_FILE_PATH 변수에 저장한다.

train 데이터와, test 데이터 path 및 파일 이름을 현재 디렉터리의 "train.csv"와 "test.csv"로 설정하여 각 변수에 저장한다.

pandas package를 이용해서 csv 파일을 읽어 train_df와 test_df에 각각 저장한다.
train 데이터와 test 데이터를 합쳐서 all_df에 저장한다.
이후에 소개할 get_preprocessed_dataset_1~6 함수들을 통해 데이터 포멧을 정리하여 all_df 변수에 저장한다.

train_X 변수에 전처리된 all_df의 Survived 열의 행에 값이 존재하는 것들만, 열을 버리고 feature들만 train_X에 저장한다.
train_y에 train_df의 Survivled 값을 저장한다.
test_X에 전처리 된 all_df의 Survived 열의 각 행들이 값이 존재하지 않으면 버린다.
요약하면 pandas로 얻어온 데이터를 Tensor로 저장한다.

위에서 만든 TitanicDataset 클래스를 만들어 초기화를 해준다.
그리고 0.8%는 train dataset으로, 0.2%는 validation dataset으로 random으로 분할해준다.

위에서 만든 TitinaicTestDataset 클래스를 test dataset으로 초기화한다.

train dataset과, validation dataset, test dataset를 함수의 반환 값으로 리턴해준다.  





In [82]:
def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df

all_df 배열에서 Pclass, Fare 열만 갖고 와서, groupby 함수를 통해서 Pclass에 맞게 값들을 배열 형태로 저장하고 mean으로 평균값을 구한다.

Fare_mean의 column의 이름을 Pclass와 Fare_mean으로 변경한다.
(merge는 공통 행에 합쳐준다면, concat은 이어 붙이기만 한다.)
후에 all_df에 Pclass의 왼쪽에 행(Fare_mean)을 추가해준다.

만약 Fare에 값이 null이면 Fare_mean의 값으로 채워준다.

즉, Pclass를 승객 요금의 평균을 이용해서 비워져있는 칸을 채운다.


In [83]:
def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

,과 .을 기준으로 최대 2번까지 나눈다.
이를 통해서 이름과, 성, Mr(Miss)로 경칭을 구분할 수 있다.

따라서 이렇게 나눈것을 family_name(성), honorific(경칭), name(이름)으로 열을 만들 수 있고,
각 column의 이름을 변경해준다.
name_df에 각 열에 맞게 데이터를 채워주고, 양 옆 공백은 없에준다.
후에 all_df의 2번째 열에 all_df와 name_df를 합쳐셔 반환해준다.

In [84]:

def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df

all_df에서 honorific과 Age를 뽑고, honorific의 기준으로 그룹화하여, Age의 중간 값을 구한다.
이를 all_df에 merge하는데, honorific을 기준으로 왼쪽에 각 경칭 별로 나이 값이 없는 열에 해당 경칭의 일반적인 나이를 추가해준다.
그리고 쓸모를 다한 경칭 별 나이 평균을 없에고 반환해준다.

즉, 경칭을 이용하여 나이 값이 들어있지 않은 행에 나이를 넣어준다. 

In [85]:
def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

SibSp와 Parch는 동승한 자매/배우자의 수, 동승한 부모 / 자식의 수 이므로 이를 더해서 가족 수로 데이터를 합쳤다.
그리고 all_df에서 family_num이 0이면, "alone"에 1을 채워주고,
만약 값이 안채운 행이 있으면 0으로 채워준다.

그리고 학습에 필요없다고 판단한, PasssengerId와 Name, family_name, name, Ticket, Cabin 열을 지워준다.

In [86]:
def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df


경칭이 Mr, Miss, Mrs, Master가 아닌 경우 other 값으로 채워준다.
그리고 Embarked(도착지)에 아무것도 안적혀있는 데이터의 경우 missing으로 표기한다.

In [87]:
def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
          le = le.fit(all_df[category_feature])
          all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df

object 타입을 가진 카테고리 변수들은 LabelEncorder를 이용해서 정수 값으로 변환한다.
그 이유는 수치값이 학습에 적절한 형태이기 때문이다.

각각의 카테고리별로 진행되어야 하므로, 각 카테고리 별로 Encorder를 만들어서 채워준다.

In [88]:
from torch import nn
class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    self.model = nn.Sequential(
      nn.Linear(n_input, 30),
      nn.ReLU(),
      nn.Linear(30, 30),
      nn.ReLU(),
      nn.Linear(30, n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x

학습할 네트워크를 만들어준다.
nn.Module을 상속하여 네트워크를 만들며, FNN과 ReLU를 이용하며 hidden layer는 2개고, output layer는 1개로 이루어진 네트워크이다.

In [89]:

def test(test_data_loader):
  print("[TEST]")
  batch = next(iter(test_data_loader))
  print("{0}".format(batch['input'].shape))
  my_model = MyModel(n_input=11, n_output=2)
  output_batch = my_model(batch['input'])
  prediction_batch = torch.argmax(output_batch, dim=1)
  for idx, prediction in enumerate(prediction_batch, start=892):
      print(idx, prediction.item())


이는 학습된 네트워크를 잘 학습됐는지 확인하기 위해 테스트를 진행하여, 결과 값을 출력한다.
input feature가 11이고, output은 2로 살았는지 죽었는지를 각각의 확률로 출력한다.

prediction_batch = torch.argmax(output_batch, dim=1)
를 통해서 더 높은 확률이 있는 index를 반환한다.

In [90]:

if __name__ == "__main__":
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

  print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
    len(train_dataset), len(validation_dataset), len(test_dataset)
  ))
  print("#" * 50, 1)

  for idx, sample in enumerate(train_dataset):
    print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

  print("#" * 50, 2)

  train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

  print("[TRAIN]")
  for idx, batch in enumerate(train_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("[VALIDATION]")
  for idx, batch in enumerate(validation_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("#" * 50, 3)

  test(test_data_loader)


train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 2.0000,  0.0000, 33.0000,  1.0000,  2.0000, 27.7500,  2.0000, 21.1792,
         3.0000,  3.0000,  0.0000]): 1
1 - tensor([ 1.0000,  0.0000, 35.0000,  1.0000,  0.0000, 53.1000,  2.0000, 87.5090,
         3.0000,  1.0000,  0.0000]): 1
2 - tensor([ 1.0000,  1.0000, 48.0000,  0.0000,  0.0000, 26.5500,  2.0000, 87.5090,
         2.0000,  0.0000,  1.0000]): 1
3 - tensor([ 3.0000,  0.0000, 22.0000,  1.0000,  0.0000, 15.5000,  1.0000, 13.3029,
         1.0000,  1.0000,  0.0000]): 1
4 - tensor([  1.0000,   1.0000,  19.0000,   3.0000,   2.0000, 263.0000,   2.0000,
         87.5090,   2.0000,   5.0000,   0.0000]): 0
5 - tensor([ 3.0000,  1.0000, 21.0000,  0.0000,  0.0000,  7.9250,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
6 - tensor([ 2.0000,  1.0000, 27.0000,  0.0000,  0.0000, 13.0000,  2.0000, 21.1792,
         4.0000,  0.0000,  1.0000]): 0
7 - te

먼저 train과, validation, test의 데이터셋 크기를 반환해준다.
그리고 train dataset에 대해서 입력될 데이터를 index와 input 데이터, target 데이터 순으로 출력해준다.

DataLoader 클래스를 이용해서 각 데이터셋을 batch 단위로 자르며, 랜덤으로 섞어서 데이터를 불러올 수 있게 만든다.
train에 사용될 데이터와 target의 shape를 반환한다.
torch의 axis=0은 batch에 들어간 데이터의 크기이며, axis=1은 feature의 수이다.

마찬가지로 validation 데이터 셋에 대해서 데이터 포멧을 출력하고, test를 진행한다.

### 요구사항 2. titanic 딥러닝 모델 훈련 코드 및 Activation	Function 변경해보기

In [103]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from datetime import datetime
import wandb
import argparse

from pathlib import Path

BASE_PATH = str(Path("__file__").resolve().parent.parent.parent)

import sys

sys.path.append(BASE_PATH)

import _02_homeworks._02_fcn_dl.titanic.titanic_dataset as td

class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    self.model = nn.Sequential(
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      nn.ReLU(),
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      nn.ReLU(),
      nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x

def get_model_and_optimizer():
    my_model = MyModel(n_input=11, n_output=2)
    optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

    return my_model, optimizer

def training_loop(model, optimizer, train_data_loader, validation_data_loader):
    n_epochs = wandb.config.epochs
    loss_fn = nn.CrossEntropyLoss()
    next_print_epoch = 100

    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        num_trains = 0
        for train_batch in train_data_loader:
            output_train = model(train_batch['input'])
            loss = loss_fn(output_train, train_batch['target'])
            loss_train += loss.item()
            num_trains += 1

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        loss_validation = 0.0
        num_validations = 0
        with torch.no_grad():
            for validation_batch in validation_data_loader:
                output_validation = model(validation_batch['input'])
                loss = loss_fn(output_validation, validation_batch['target'])
                loss_validation += loss.item()
                num_validations += 1

        wandb.log({
            "Epoch": epoch,
            "Training loss": loss_train / num_trains,
            "Validation loss": loss_validation / num_validations
        })

        if epoch >= next_print_epoch:
            print(
                f"Epoch {epoch}, "
                f"Training loss {loss_train / num_trains:.4f}, "
                f"Validation loss {loss_validation / num_validations:.4f}"
            )
            next_print_epoch += 100

def main(args):
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': 1e-3,
        'n_hidden_unit_list': [20, 20],
    }

    wandb.init(
        mode="online" if args.wandb else "disabled",
        project="HW2_2019136011",
        notes="HW2_2019136011",
        tags=["titanic_dataset", "HW2_2019136011"],
        name=current_time_str,
        config=config
    )
    print(args)
    print(wandb.config)

    train_dataset, validation_dataset, test_dataset = td.get_preprocessed_dataset()
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=args.batch_size, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=args.batch_size, shuffle=True)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    linear_model, optimizer = get_model_and_optimizer()

    wandb.watch(linear_model)

    print("#" * 50, 1)

    training_loop(
        model=linear_model,
        optimizer=optimizer,
        train_data_loader=train_data_loader,
        validation_data_loader=validation_data_loader
    )
    wandb.finish()


# https://docs.wandb.ai/guides/track/config
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    
    parser.add_argument(
        "--wandb", action=argparse.BooleanOptionalAction, default=False, help="True or False"
    )

    parser.add_argument(
        "-b", "--batch_size", type=int, default=512, help="Batch size (int, default: 512)"
    )

    parser.add_argument(
        "-e", "--epochs", type=int, default=1_000, help="Number of training epochs (int, default:1_000)"
    )
    
    parser.add_argument(
        "-f", "--file", type=str, default="for ipynb", help="for ipynb"
    )

    args = parser.parse_args()

    main(args)



Namespace(wandb=False, batch_size=512, epochs=1000, file='C:\\Users\\KwonSungMin\\AppData\\Roaming\\jupyter\\runtime\\kernel-abe3dcc1-f9ba-4087-a9d2-85568eec5f3c.json')
{'epochs': 1000, 'batch_size': 512, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20]}
################################################## 1
Epoch 100, Training loss 0.5760, Validation loss 0.6026
Epoch 200, Training loss 0.5587, Validation loss 0.6057
Epoch 300, Training loss 0.5531, Validation loss 0.6087
Epoch 400, Training loss 0.5572, Validation loss 0.6077
Epoch 500, Training loss 0.5644, Validation loss 0.6079
Epoch 600, Training loss 0.5752, Validation loss 0.6081
Epoch 700, Training loss 0.5516, Validation loss 0.6074
Epoch 800, Training loss 0.5568, Validation loss 0.6071
Epoch 900, Training loss 0.5569, Validation loss 0.6090
Epoch 1000, Training loss 0.5619, Validation loss 0.6087


살아있는지 여부를 classification 하는 문제이기 때문에 CrossEntropyLoss를 사용하였다. 

#### 수행한 Activation Function List
* PReLU
* Sigmoid
* ReLU6
* ReLU
* Tanh
* ELU
* LeakyReLU

![](./lossGraph.png)


wandb link : https://wandb.ai/00kwonsm/HW2_2019136011/workspace?workspace=user-00kwonsm

epoch 10,000 번 실행 결과, 
Validation loss에서는 LeakyReLU가 가장 좋은 결과가 나온 반면에, Training loss에서는 ELU 가장 좋은 결과가 나왔음.

### [요구사항 3] 테스트 및 submission.csv 생성 

Training loss는 과적합이 일어난 것일 수 있으므로, 선택한 Activation Function은 LeakyReLU이다.

In [107]:
# ...

def training_loop(model, optimizer, train_data_loader, validation_data_loader, test_data_loader):
    n_epochs = wandb.config.epochs
    loss_fn = nn.CrossEntropyLoss()
    next_print_epoch = 100
    best_validation = 1.0

    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        num_trains = 0
        for train_batch in train_data_loader:
            output_train = model(train_batch['input'])
            loss = loss_fn(output_train, train_batch['target'])
            loss_train += loss.item()
            num_trains += 1

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        loss_validation = 0.0
        num_validations = 0
        with torch.no_grad():
            for validation_batch in validation_data_loader:
                output_validation = model(validation_batch['input'])
                loss = loss_fn(output_validation, validation_batch['target'])
                loss_validation += loss.item()
                num_validations += 1

        wandb.log({
            "Epoch": epoch,
            "Training loss": loss_train / num_trains,
            "Validation loss": loss_validation / num_validations
        })

        if epoch >= next_print_epoch:
            print(
                f"Epoch {epoch}, "
                f"Training loss {loss_train / num_trains:.4f}, "
                f"Validation loss {loss_validation / num_validations:.4f}"
            )
            next_print_epoch += 100

        if(best_validation > loss_train / num_trains):
            best_validation = loss_train / num_trains
            test(my_model=model, test_data_loader=test_data_loader)

def main(args):
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': 1e-3,
        'n_hidden_unit_list': [20, 20],
    }

    wandb.init(
        mode="online" if args.wandb else "disabled",
        project="HW2_2019136011",
        notes="HW2_2019136011",
        tags=["titanic_dataset", "HW2_2019136011"],
        name=current_time_str,
        config=config
    )
    print(args)
    print(wandb.config)

    train_dataset, validation_dataset, test_dataset = td.get_preprocessed_dataset()
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=args.batch_size, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=args.batch_size, shuffle=True)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    linear_model, optimizer = get_model_and_optimizer()

    wandb.watch(linear_model)

    print("#" * 50, 1)

    training_loop(
        model=linear_model,
        optimizer=optimizer,
        train_data_loader=train_data_loader,
        validation_data_loader=validation_data_loader,
        test_data_loader=test_data_loader
    )
    wandb.finish()

def test(test_data_loader, my_model):
  # print("[TEST]")
  batch = next(iter(test_data_loader))
  # print("{0}".format(batch['input'].shape))
  output_batch = my_model(batch['input'])
  prediction_batch = torch.argmax(output_batch, dim=1)

  import csv
  with open('submission.csv', 'w', newline='') as file:
      writer = csv.writer(file)
      header = [str("PassengerId"),str("Survived")]
      writer.writerow(header)
      for idx, prediction in enumerate(prediction_batch, start=892):
          data = [str(idx),str(prediction.item())]
          writer.writerow(data)


# ...          

submission.csv를 출력하기 위해서, import csv를 진행한 후에, 각 test 결과에 대해서 출력을 진행하면 된다.

Test가 진행되는 시점은 validation loss가 가장 적은 시점이 가장 좋은 경우이기 때문에 가장 낮을 때 결과를 출력한다.

### [요구사항 4] submission.csv 제출 및 등수확인

![](./Kaggle_Competition.png)

### [요구사항 5] Wandb 페이지 생성 및 URL 제출

https://wandb.ai/00kwonsm/HW2_2019136011?workspace=user-00kwonsm

### 숙제 후기

이번 과제를 통해서 loss function을 어떤 것을 선택해야 되는지 알 수 있었으며, 각 activation function에 따라서 어떠한 결과를 나타내는지 알 수 있었다.
특히, activation function의 결과가 validation과 train loss의 결과가 크게 달라지는 점이 재미있었다.
즉, 항상 좋은 activation function이란 없음을 깨달을 수 있었다.